# 🔑 Password Strength Visualizer + Explainer

## Overview

This project is an interactive **Password Strength Visualizer** that analyzes user passwords, visualizes their security properties, and provides real-time, AI-powered feedback. It helps users understand why their passwords are strong or weak, making security concepts accessible through engaging visualizations and explanations.

## Features

- **Entropy Calculation:** Measures how unpredictable a password is.
- **Guessing Time Estimation:** Shows how long it would take to crack the password under various attack scenarios.
- **Dictionary Attack Simulation:** Checks if the password is found in common password lists.
- **Real-Time Visual Feedback:** Uses charts and progress bars for instant analysis.
- **AI-Powered Explanations:** Explains password strengths and weaknesses using a lightweight language model.
- **User-Friendly Interface:** Built with Gradio for ease of use and interactivity.

## What I Implemented

- Developed a core password analysis engine using information theory and the zxcvbn library.
- Integrated AI explanations for password quality using a small language model.
- Created interactive visualizations (entropy gauge, attack time bar chart, character type pie chart).
- Built a web interface with Gradio for real-time password analysis and feedback.
- Ensured privacy by not storing passwords and using secure input fields.
- Provided educational feedback and actionable suggestions to improve password strength.

## Technologies Used

| Component                | Technology/Library          |
|--------------------------|----------------------------|
| Password Analysis        | Python, zxcvbn             |
| Entropy & Attack Models  | math, information theory   |
| AI Explanations          | transformers, DialoGPT     |
| Visualization            | Plotly, Matplotlib, Seaborn|
| Web Interface            | Gradio                     |
| Data Handling            | pandas, numpy              |


In [ ]:
# Install required packages
!pip install zxcvbn-python gradio transformers torch matplotlib seaborn plotly
!pip install wordlist

# Import necessary libraries
import math
import time
import re
import string
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
from zxcvbn import zxcvbn
import gradio as gr
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import warnings
warnings.filterwarnings('ignore')

print("✅ All packages installed successfully!")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.0/408.0 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 33.8 MB/s eta 0:00:00
  Created wheel for zxcvbn-python: filename=zxcvbn_python-4.4.24-py3-none-any.whl

In [ ]:
## Step 2: Password Entropy Calculation
class PasswordAnalyzer:
    def __init__(self):
        self.common_passwords = self.load_common_passwords()

    def calculate_entropy(self, password):
        """
        Calculate password entropy using Shannon entropy formula
        Entropy = log2(N^L) where N = character set size, L = password length
        """
        if not password:
            return 0

        # Determine character set size
        charset_size = 0

        if any(c in string.ascii_lowercase for c in password):
            charset_size += 26  # lowercase letters
        if any(c in string.ascii_uppercase for c in password):
            charset_size += 26  # uppercase letters
        if any(c in string.digits for c in password):
            charset_size += 10  # numbers
        if any(c in string.punctuation for c in password):
            charset_size += 32  # special characters

        # Calculate entropy
        entropy = len(password) * math.log2(charset_size) if charset_size > 0 else 0
        return round(entropy, 2)

    def estimate_guessing_time(self, password):
        """
        Estimate time to guess password using different attack methods
        """
        entropy = self.calculate_entropy(password)

        # Different attack speeds (guesses per second)
        attack_speeds = {
            'Online Attack (Slow)': 1000,  # 1K guesses/sec
            'Online Attack (Fast)': 100000,  # 100K guesses/sec
            'Offline Attack (GPU)': 1000000000,  # 1B guesses/sec
            'Offline Attack (Supercomputer)': 100000000000  # 100B guesses/sec
        }

        results = {}
        total_combinations = 2 ** entropy

        for attack_type, speed in attack_speeds.items():
            # Average time is half the total search space
            avg_time_seconds = (total_combinations / 2) / speed
            results[attack_type] = self.format_time(avg_time_seconds)

        return results

    def format_time(self, seconds):
        """Convert seconds to human-readable format"""
        if seconds < 60:
            return f"{seconds:.2f} seconds"
        elif seconds < 3600:
            return f"{seconds/60:.2f} minutes"
        elif seconds < 86400:
            return f"{seconds/3600:.2f} hours"
        elif seconds < 31536000:
            return f"{seconds/86400:.2f} days"
        else:
            return f"{seconds/31536000:.2f} years"

    def load_common_passwords(self):
        """Load common passwords for dictionary attack simulation"""
        # This is a simplified list - in practice, you'd load from a file
        return [
            'password', '123456', 'password123', 'admin', 'qwerty',
            'letmein', 'welcome', 'monkey', '1234567890', 'abc123',
            'Password1', 'iloveyou', 'princess', 'rockyou', 'abc123'
        ]

    def check_dictionary_attack(self, password):
        """Check if password exists in common password dictionary"""
        return password.lower() in [p.lower() for p in self.common_passwords]

# Initialize analyzer
analyzer = PasswordAnalyzer()
print("✅ Password Analyzer initialized!")


✅ Password Analyzer initialized!


In [ ]:
## Step 3: Advanced Password Analysis with zxcvbn
def advanced_password_analysis(password):
    """
    Use zxcvbn for advanced password analysis
    """
    if not password:
        return None

    # Run zxcvbn analysis
    result = zxcvbn(password)

    analysis = {
        'score': result['score'],  # 0-4 scale
        'entropy': analyzer.calculate_entropy(password),
        'crack_times': result['crack_times_display'],
        'feedback': result['feedback'],
        'pattern_matches': len(result['sequence']),
        'guesses': result['guesses']
    }

    return analysis

def get_strength_color(score):
    """Return color based on password strength score"""
    colors = {
        0: '#ff4444',  # Very Weak - Red
        1: '#ff8800',  # Weak - Orange
        2: '#ffbb00',  # Fair - Yellow
        3: '#88cc00',  # Good - Light Green
        4: '#00cc44'   # Strong - Green
    }
    return colors.get(score, '#cccccc')

def get_strength_label(score):
    """Return label based on password strength score"""
    labels = {
        0: 'Very Weak',
        1: 'Weak',
        2: 'Fair',
        3: 'Good',
        4: 'Strong'
    }
    return labels.get(score, 'Unknown')

print("✅ Advanced analysis functions ready!")


✅ Advanced analysis functions ready!


In [ ]:
## Step 4: AI-Powered Password Explanation
# Initialize a lightweight language model for explanations
try:
    # Try to load a small, efficient model
    model_name = "microsoft/DialoGPT-small"  # Lightweight alternative to Tiny LLaMA
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)

    # Create text generation pipeline
    text_generator = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_length=150,
        num_return_sequences=1,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    ai_available = True
    print("✅ AI model loaded successfully!")

except Exception as e:
    print(f"⚠️ Could not load AI model: {e}")
    print("Will use rule-based explanations instead.")
    ai_available = False

def generate_password_explanation(password, analysis):
    """
    Generate human-readable explanation of password strength
    """
    if not analysis:
        return "Unable to analyze password."

    score = analysis['score']
    entropy = analysis['entropy']
    feedback = analysis['feedback']

    # Rule-based explanation (fallback)
    explanations = []

    if score <= 1:
        explanations.append(f"This password is {get_strength_label(score).lower()} with only {entropy:.1f} bits of entropy.")
        if feedback['warning']:
            explanations.append(f"Warning: {feedback['warning']}")
        if feedback['suggestions']:
            explanations.append(f"Suggestions: {'. '.join(feedback['suggestions'])}")

    elif score == 2:
        explanations.append(f"This password is {get_strength_label(score).lower()} with {entropy:.1f} bits of entropy.")
        explanations.append("Consider adding more character variety and length.")

    elif score >= 3:
        explanations.append(f"This password is {get_strength_label(score).lower()} with {entropy:.1f} bits of entropy.")
        explanations.append("Good job! This password should resist most attacks.")

    base_explanation = " ".join(explanations)

    # Try AI enhancement if available
    if ai_available and len(password) > 0:
        try:
            prompt = f"Explain why this password is {get_strength_label(score).lower()}: "
            response = text_generator(prompt, max_length=100, num_return_sequences=1)
            ai_explanation = response[0]['generated_text'].replace(prompt, "").strip()

            if len(ai_explanation) > 10:  # Basic quality check
                return f"{base_explanation}\n\nAI Insight: {ai_explanation}"
        except:
            pass

    return base_explanation

print("✅ AI explanation system ready!")


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/641 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/351M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cpu


✅ AI model loaded successfully!
✅ AI explanation system ready!


In [ ]:
##  Step 5: Visualization Functions
def create_entropy_gauge(entropy):
    """Create a gauge chart for password entropy"""
    fig = go.Figure(go.Indicator(
        mode = "gauge+number+delta",
        value = entropy,
        domain = {'x': [0, 1], 'y': [0, 1]},
        title = {'text': "Password Entropy (bits)"},
        delta = {'reference': 50},
        gauge = {
            'axis': {'range': [None, 100]},
            'bar': {'color': "darkblue"},
            'steps': [
                {'range': [0, 25], 'color': "lightgray"},
                {'range': [25, 50], 'color': "yellow"},
                {'range': [50, 75], 'color': "orange"},
                {'range': [75, 100], 'color': "green"}
            ],
            'threshold': {
                'line': {'color': "red", 'width': 4},
                'thickness': 0.75,
                'value': 60
            }
        }
    ))

    fig.update_layout(height=300)
    return fig

def create_strength_visualization(analysis):
    """Create comprehensive strength visualization"""
    if not analysis:
        return None

    score = analysis['score']
    entropy = analysis['entropy']

    # Create subplots
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Password Strength Score', 'Entropy Level',
                       'Attack Time Estimates', 'Character Analysis'),
        specs=[[{"type": "indicator"}, {"type": "indicator"}],
               [{"type": "bar"}, {"type": "pie"}]]
    )

    # Strength score indicator
    fig.add_trace(go.Indicator(
        mode = "gauge+number",
        value = score,
        title = {'text': f"Score: {get_strength_label(score)}"},
        gauge = {
            'axis': {'range': [0, 4]},
            'bar': {'color': get_strength_color(score)},
            'steps': [{'range': [0, 4], 'color': "lightgray"}]
        }
    ), row=1, col=1)

    # Entropy indicator
    fig.add_trace(go.Indicator(
        mode = "gauge+number",
        value = entropy,
        title = {'text': "Entropy (bits)"},
        gauge = {
            'axis': {'range': [0, 100]},
            'bar': {'color': "blue"},
            'steps': [{'range': [0, 100], 'color': "lightgray"}]
        }
    ), row=1, col=2)

    # Attack time estimates
    guessing_times = analyzer.estimate_guessing_time(analysis.get('password', ''))
    attack_types = list(guessing_times.keys())
    # Convert times to numeric values for visualization (simplified)
    time_values = [len(str(time)) for time in guessing_times.values()]  # Rough approximation

    fig.add_trace(go.Bar(
        x=attack_types,
        y=time_values,
        name="Attack Times",
        marker_color=['red', 'orange', 'yellow', 'green']
    ), row=2, col=1)

    # Character type analysis
    char_types = ['Lowercase', 'Uppercase', 'Numbers', 'Symbols']
    char_counts = [0, 0, 0, 0]

    if 'password' in analysis:
        password = analysis['password']
        char_counts[0] = sum(1 for c in password if c.islower())
        char_counts[1] = sum(1 for c in password if c.isupper())
        char_counts[2] = sum(1 for c in password if c.isdigit())
        char_counts[3] = sum(1 for c in password if c in string.punctuation)

    fig.add_trace(go.Pie(
        labels=char_types,
        values=char_counts,
        name="Character Types"
    ), row=2, col=2)

    fig.update_layout(height=600, title_text="Password Strength Analysis")
    return fig

print("✅ Visualization functions ready!")


✅ Visualization functions ready!


In [ ]:
## Step 6: Gradio Interface
def analyze_password_complete(password):
    """
    Complete password analysis function for Gradio interface
    """
    if not password:
        return (
            "Please enter a password to analyze.",
            None,
            "No password provided.",
            "Enter a password to see detailed analysis."
        )

    # Perform analysis
    analysis = advanced_password_analysis(password)
    analysis['password'] = password  # Add password to analysis for visualization

    # Generate explanation
    explanation = generate_password_explanation(password, analysis)

    # Create visualization
    viz_fig = create_strength_visualization(analysis)

    # Create summary
    if analysis:
        score = analysis['score']
        entropy = analysis['entropy']
        dictionary_hit = analyzer.check_dictionary_attack(password)

        summary = f"""
## Password Analysis Summary

**Strength Score:** {score}/4 ({get_strength_label(score)})
**Entropy:** {entropy:.2f} bits
**Dictionary Attack Vulnerable:** {'⚠️ Yes' if dictionary_hit else '✅ No'}

### Estimated Crack Times:
"""
        guessing_times = analyzer.estimate_guessing_time(password)
        for attack_type, time_estimate in guessing_times.items():
            summary += f"\n- **{attack_type}:** {time_estimate}"

        if analysis['feedback']['warning']:
            summary += f"\n\n⚠️ **Warning:** {analysis['feedback']['warning']}"

        if analysis['feedback']['suggestions']:
            summary += f"\n\n💡 **Suggestions:**"
            for suggestion in analysis['feedback']['suggestions']:
                summary += f"\n- {suggestion}"

    else:
        summary = "Unable to analyze password."

    return summary, viz_fig, explanation, "Analysis complete!"

# Create Gradio interface
def create_gradio_interface():
    """Create the main Gradio interface"""

    with gr.Blocks(title="🔑 Password Strength Visualizer", theme=gr.themes.Soft()) as demo:
        gr.Markdown("""
        # 🔑 Password Strength Visualizer + AI Explainer

        Enter a password to get comprehensive security analysis with visualizations and AI-powered explanations.

        **Features:**
        - 🔢 Entropy calculation and visualization
        - ⏱️ Attack time estimation across different scenarios
        - 📚 Dictionary attack vulnerability check
        - 🤖 AI-powered strength explanations
        - 📊 Interactive charts and gauges
        """)

        with gr.Row():
            with gr.Column(scale=1):
                password_input = gr.Textbox(
                    label="Enter Password",
                    placeholder="Type your password here...",
                    type="password",
                    lines=1
                )

                analyze_btn = gr.Button("🔍 Analyze Password", variant="primary")

                # Real-time analysis toggle
                realtime_check = gr.Checkbox(
                    label="Real-time analysis",
                    value=False,
                    info="Analyze as you type"
                )

            with gr.Column(scale=2):
                status_output = gr.Textbox(
                    label="Status",
                    value="Ready for analysis...",
                    interactive=False
                )

        with gr.Row():
            with gr.Column():
                summary_output = gr.Markdown(label="Analysis Summary")

            with gr.Column():
                explanation_output = gr.Textbox(
                    label="🤖 AI Explanation",
                    lines=5,
                    interactive=False
                )

        # Visualization area
        plot_output = gr.Plot(label="📊 Security Visualization")

        # Set up event handlers
        analyze_btn.click(
            fn=analyze_password_complete,
            inputs=[password_input],
            outputs=[summary_output, plot_output, explanation_output, status_output]
        )

        # Real-time analysis (optional)
        password_input.change(
            fn=lambda pwd, realtime: analyze_password_complete(pwd) if realtime and pwd else ("", None, "", ""),
            inputs=[password_input, realtime_check],
            outputs=[summary_output, plot_output, explanation_output, status_output]
        )

        # Example passwords section
        gr.Markdown("### 🧪 Try These Example Passwords:")
        example_passwords = [
            "password123",  # Weak
            "MyP@ssw0rd2024!",  # Strong
            "qwerty",  # Very weak
            "Tr0ub4dor&3"  # Strong with pattern
        ]

        with gr.Row():
            for pwd in example_passwords:
                gr.Button(pwd, size="sm").click(
                    lambda p=pwd: p,
                    outputs=password_input
                )

    return demo

# Launch the interface
demo = create_gradio_interface()
print("✅ Gradio interface created!")


✅ Gradio interface created!


In [ ]:
# Launch the application
if __name__ == "__main__":
    print("🚀 Launching Password Strength Visualizer...")
    print("📊 Features enabled:")
    print("   ✅ Entropy calculation")
    print("   ✅ Attack time estimation")
    print("   ✅ Dictionary attack check")
    print("   ✅ Interactive visualizations")
    print(f"   {'✅' if ai_available else '⚠️'} AI explanations")
    print("   ✅ Real-time analysis")

    # Launch with sharing enabled for Colab
    demo.launch(
        share=True,  # Creates public link for Colab
        debug=True,  # Enable debug mode
        server_name="0.0.0.0",  # Allow external connections
        server_port=7860  # Default Gradio port
    )


🚀 Launching Password Strength Visualizer...
📊 Features enabled:
   ✅ Entropy calculation
   ✅ Attack time estimation
   ✅ Dictionary attack check
   ✅ Interactive visualizations
   ✅ AI explanations
   ✅ Real-time analysis
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://7fe21a5f67dd6199ff.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
